In [19]:
# library imports
import pandas as pd
import numpy as np
import os
import glob

In [20]:
def check_duplicate_cols(file_path):
    '''
    Input: csv file path
    Checks if the csv file have duplicate column names.
    '''
    df = pd.read_csv(file_path,encoding="utf-8")

    # Get column names and find duplicates
    column_names = df.columns
    duplicate_columns = column_names[column_names.duplicated()].tolist()

    if duplicate_columns:
        print(f"WARNING! Duplicate column titles found: {duplicate_columns}")
    else:
        print("\nNice! No duplicate column titles found.\n")

In [21]:
def get_columns_from_formatted_codebook(file_path_formatted_codebook, verbose):
    '''
    input: formatted codebook filepath
    Read the formatted codebook csv to get the questions and their color categories
    '''
    # Read the dataset from file
    df = pd.read_csv(file_path_formatted_codebook)

    # Creating no_color_questions for rows where 'color' is empty
    no_color_questions = df.loc[df['Color_Category'].isnull(), 'Custom_variable_name'].tolist()

    # Creating lists for each color, where grey_questions=question having sub categories.
    grey_questions  = df.loc[df['Color_Category'] == 'Grey', 'Custom_variable_name'].tolist()
    yellow_questions = df.loc[df['Color_Category'] == 'Yellow', 'Custom_variable_name'].tolist()
    green_questions  = df.loc[df['Color_Category'] == 'Green', 'Custom_variable_name'].tolist()

    # Output the results if needed
    if(verbose):
        print("::::get_columns_from_formatted_codebook:::::Returns")
        print("no_color_questions:", no_color_questions)
        print("grey_questions(question having sub categories):", grey_questions )
        print("yellow_questions:", yellow_questions )
        print("green_questions:", green_questions )
    return no_color_questions, grey_questions,yellow_questions, green_questions

In [22]:
def filter_datasets_for_required_col(file_path_dataset,file_path_formatted_sosec_code_book, columns_to_keep, verbose=False):
    '''
    input:codebook file path and sosec dataset file path
    Only Keeps the columns in the dataset which are also in the codebook and saves the files
    also Saves a file for the columns in the code book which are not in the dataset
    '''
    # Read the dataset CSV file
    df = pd.read_csv(file_path_dataset)

    # Check which columns are not found
    not_found_columns = [col for col in columns_to_keep if col not in df.columns]

    if not_found_columns:
        # Read the formatted_sosec_code_book CSV file
        file_path_2 = file_path_formatted_sosec_code_book
        df2 = pd.read_csv(file_path_2,encoding="utf-8")
        
        # Filter rows where Custom_variable_name matches values in not_found_columns
        df_no_matches = df2[df2['Custom_variable_name'].isin(not_found_columns)]
        
        # Select the corresponding Text values
        result_df = df_no_matches[['Custom_variable_name', 'Text']]
        
        # Save the result as a new CSV
        output_file_path = r'../data/1_preprocess/1_codebook_no_matching_columns_in_dataset.csv'
        result_df.to_csv(output_file_path, index=False)
        if(verbose):
            print(f"Not Matched columns saved to: {output_file_path}")

    # Filter the DataFrame to keep only the columns that exist in the DataFrame
    df_filtered = df[[col for col in columns_to_keep if col in df.columns]]
    
    output_file_path = r'../data/1_preprocess/1_df_dataset_with_codebook_columns_full_no_processing.csv'
    df_filtered.to_csv(output_file_path, index=False)

    if(verbose):
        print(df_filtered.head())
    
    return output_file_path+""

In [23]:
def get_row_count(file_path_csv):
    '''
    input filepath of csv
    Returns the number of rows in the given CSV file.
    '''
    # Load the dataset
    df = pd.read_csv(file_path_csv)

    # Get the number of rows
    row_count = len(df)

    print(f"Number of rows in the csv: {row_count}")
    return row_count

In [24]:
def count_columns_in_csv(file_path_csv):
    """
    Counts and returns the number of columns in a CSV file.
    
    Parameters:
    file_path (str): The path to the CSV file.
    
    Returns:
    int: The number of columns in the CSV file.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path_csv)
    
    # Get the number of col
    columns_count = len(df.columns)
    
    
    print(f"Number of columns in the CSV: {columns_count}")
    # Return the number of columns
    return columns_count

In [25]:
def perform_filter_on_dataset_F2A3(file_path_dataset_with_codebook_columns):
    '''
    input the datafile path with only codebook columns
    replace 6 and 0 by blank
    F2A3 = The possibility of losing your job (leave this empty if you do not work)
    '''
    # Load the dataset
    df = pd.read_csv(file_path_dataset_with_codebook_columns)

    # Replace 6 and 0 in the 'F2A3' column with  empty
    df['F2A3'] = df['F2A3'].replace({6: "", 0: ""})

    # Save the updated DataFrame back to a CSV file
    output_file_path = r'../data/1_preprocess/2_df_dataset_with_codebook_columns_full_F2A3.csv'
    df.to_csv(output_file_path, index=False)

    print(f"Updated dataset saved to: {output_file_path}")
    return output_file_path+""

In [26]:
def perform_filter_on_dataset_F7mA1(file_path_dataset_with_codebook_columns):
    '''
    input the datafile path with only codebook columns
    replace 0 by blank
    F7mA1 = job category (leave this empty if you do not work)
    '''
    # Load the dataset
    df = pd.read_csv(file_path_dataset_with_codebook_columns)

    # Replace 0 in the 'F7mA1' column with  empty
    df['F7mA1'] = df['F7mA1'].replace({0: ""})

    # Save the updated DataFrame back to a CSV file
    output_file_path = r'../data/1_preprocess/3_df_dataset_with_codebook_columns_full_F7mA1.csv'
    df.to_csv(output_file_path, index=False)

    print(f"Updated dataset saved to: {output_file_path}")
    return output_file_path+""

In [27]:
def delete_rows_for_out_of_range_data(sosec_data_path,file_path_dataset_with_codebook_columns,verbose=False):
    '''
    input: sosec_datafile path and sosec dataset with only codebook columns
    Deletes the rows for all the columns for with the values are out of range. and saves the csv
    '''
    file1 = pd.read_csv(sosec_data_path)

    file2 = pd.read_csv(file_path_dataset_with_codebook_columns)

    # Create a dictionary from file2 with ranges
    ranges = {}
    for _, row in file2.iterrows():
        col = row['Custom_variable_name']
        range_str = row['Characteristic']
        
        # Check if range_str is a valid string before splitting
        if isinstance(range_str, str) and range_str and range_str != 'F':  # Valid range and not 'F'
            # If there's a valid range, split it into a list of integers
            ranges[col] = list(map(int, range_str.split(',')))
        else:
            # If no valid range is provided (empty or 'F'), set the range to None
            ranges[col] = None


    # Validate the data in file1 against the ranges
    def validate_data(file1, ranges):
        errors = []
        valid_rows = file1.copy()  # Copy of the original DataFrame to modify
        
        # Loop through each column and validate values
        for col in file1.columns:
            if col in ranges:
                valid_range = ranges[col]
                if valid_range is not None:  # Only check if a valid range exists
                    # Create a boolean mask for invalid rows
                    invalid_rows = ~(valid_rows[col].isin(valid_range) | valid_rows[col].isna())      

                    # Track errors for rows with out-of-range data
                    for index, value in valid_rows[invalid_rows][col].dropna().items():
                        errors.append(f"Out of range: {col} at row {index + 1} with value {value}")
                    
                    # Remove rows with invalid data
                    valid_rows = valid_rows[~invalid_rows]
                else:
                    # If no range is provided, assume all values are valid for that column
                    continue
        
        return valid_rows, errors

    # Get valid rows and errors
    valid_rows, errors = validate_data(file1, ranges)

    # Save the valid rows to a CSV file
    output_file_path = r"../data/1_preprocess/4_df_dataset_with_codebook_columns_filtered_outofrange.csv"
    valid_rows.to_csv(output_file_path, index=False)

    if(verbose):
        # Output validation errors
        if errors:
            print("Validation Errors:")
            for error in errors:
                print(error)
        else:
            print("All data is within valid ranges.")

    # Print where the cleaned data has been saved
    print(f"Cleaned data saved to '{output_file_path}'.")

    return output_file_path+""

In [28]:
def perform_filter_on_dataset_F7cA1(file_path_dataset_with_codebook_columns,min_yob,max_yob):
    '''
    filter rows based on range for F7cA1(Yob).
    '''
    # Load the dataset
    df = pd.read_csv(file_path_dataset_with_codebook_columns)

    # Filter out rows where 'F7cA1' is not in range
    df = df[(df['F7cA1'] >= min_yob) & (df['F7cA1'] <= max_yob)]

    # Save the updated DataFrame back to a CSV file
    output_file_path = r'../data/1_preprocess/5_df_dataset_with_codebook_columns_filtered_F7cA1-yob.csv'
    df.to_csv(output_file_path, index=False)

    print(f"Updated dataset saved to: {output_file_path}")
    return output_file_path


In [29]:
def drop_high_null_columns(FILE_PATH_DATASET,file_path_formatted_sosec_code_book ,col_to_exclude, threshold=0.7 ):
    '''
    Deletes columns with more than 70% null values from file_name_A, excluding from list of columns and saves the dataset.
    Prints removed columns and saves them to a CSV file.

    Parameters:
        file_name_A (str): Path to the input CSV file.
        col_to_exclude(list): Name of the column to exclude from deletion.
        threshold (float): Proportion of nulls above which columns are dropped.
        output_file (str): Path to save the filtered dataset.
        removed_columns_file (str): Path to save the list of removed columns.
    '''

    output_file = r"../data/1_preprocess/6_df_dataset_with_codebook_columns_filtered_lessdata.csv"

    # Load the dataset
    df_A = pd.read_csv(FILE_PATH_DATASET)
    
    # Calculate the threshold for null values
    null_threshold = threshold * len(df_A)
    
    # Identify columns to keep based on null percentage and exceptions
    cols_to_keep = [col for col in df_A.columns 
                    if (df_A[col].isna().sum() <= null_threshold) or col in col_to_exclude]
    
    # Identify the columns to remove
    cols_to_remove = [col for col in df_A.columns if col not in cols_to_keep]
    
    # Print the removed columns
    print(f"Removed columns: {cols_to_remove}")
    

    if cols_to_remove:
        # Read the formatted_sosec_code_book CSV file
        file_path_2 = file_path_formatted_sosec_code_book
        df2 = pd.read_csv(file_path_2,encoding="utf-8")
        
        # Filter rows where Custom_variable_name matches values in not_found_columns
        df_no_matches = df2[df2['Custom_variable_name'].isin(cols_to_remove)]
        
        # Select the corresponding Text values
        result_df = df_no_matches[['Custom_variable_name', 'Text']]
        
        # Save the result as a new CSV
        removed_columns_file= r"../data/1_preprocess/6_removed_columns_due_to_lessdata.csv"
        result_df.to_csv(removed_columns_file, index=False)
    
        print(f"Not Matched columns saved to: {removed_columns_file}")


    # Filter the DataFrame to keep only the selected columns
    df_filtered = df_A[cols_to_keep]

    # Save the filtered dataset
    df_filtered.to_csv(output_file, index=False)
    print(f"Filtered dataset saved to: {output_file}")
    print(f"List of removed columns saved to: {removed_columns_file}")

    return output_file+""

In [30]:
def remove_below_percentile(file_path_sosec_dataset, percentile=10):
    '''
    performs filter on i_TIME using the given percentile value
    input: csv of sosec dataset.
    percentile value
    output: filtered dataset as csv
    '''
    # Load the dataset
    df = pd.read_csv(file_path_sosec_dataset)

    # Calculate the 10th percentile of the 'i_TIME' column
    percentile_value = df['i_TIME'].quantile(percentile / 100.0)

    # Filter the DataFrame to keep only rows where 'i_TIME' is greater than or equal to the 10th percentile
    df_filtered = df[df['i_TIME'] >= percentile_value]

    # Save the updated DataFrame back to a CSV file
    output_file_path = r"../data/1_preprocess/7_df_dataset_with_codebook_columns_filtered_itime.csv"
    df_filtered.to_csv(output_file_path, index=False)

    print(f"Updated dataset saved to: {output_file_path}")
    return output_file_path

In [31]:
# Function to load CSV and one-hot encode specific columns
def one_hot_encode_csv(input_file_path, columns_to_encode):
    '''
    input: csv of sosec dataset.
    list of columns to encode.
    deleted the columns_to_encode from the data.
    save the csv

    '''

    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_file_path)
    
    # Select only available columns from the list
    available_columns = [col for col in columns_to_encode if col in df.columns]

    # One-hot encode the available columns
    df_encoded = pd.get_dummies(df, columns=available_columns)

    output_file_path = r"../data/1_preprocess/8_df_dataset_with_codebook_columns_filtered_hotencoding.csv"

    # Save the encoded DataFrame to a new CSV file
    df_encoded.to_csv(output_file_path, index=False)

    print(f"One-hot encoded CSV saved to {output_file_path}")
    return output_file_path+""



In [32]:
def delete_all_csv_files(folder_path):
    """
    Deletes all CSV files in the specified folder.

    Parameters:
        folder_path (str): Relative or absolute path to the folder.
    """
    # Get the full path of all CSV files in the folder
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    if not (csv_files):
        print(f"No csv files to delete in {folder_path}")
    else:
        for file_path in csv_files:
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Failed to delete {file_path}: {e}")
    print("\n")

In [38]:
def prepare_for_personas(FILE_PATH_DATASET,FILE_PATH_REFORMATED_SOSEC_CODE_BOOK):
    # Load the files
    file1 = pd.read_csv(FILE_PATH_DATASET, sep=",")  # Adjust separator if necessary
    file2 = pd.read_csv(FILE_PATH_REFORMATED_SOSEC_CODE_BOOK, sep=",")  # Adjust separator if necessary

    # Create a mapping from file2
    mapping = dict(zip(file2['Custom_variable_name'], file2['Text']))

    # Rename columns in file1
    file1.rename(columns={"i_TIME": "i_TIME (Time taken in seconds to fill the survey)"}, inplace=True)
    file1 = file1.rename(columns=mapping)  # Assign back to file1

    # Save the updated file1
    output_file_path = r"../data/1_preprocess/9_processed_data_for_personas.csv"

    file1.to_csv(output_file_path, sep=",", index=False)

    print("Column headers replaced successfully to Questions!")

In [40]:
# Load the required data files.
FILE_PATH_DATASET = r"..\data\0_SOSEC Data RCS\data_sample_35_SOSEC_dataset_us.csv"
FILE_PATH_REFORMATED_SOSEC_CODE_BOOK = r'..\data\0_Reformated_SOSEC_Code-book_US_November.csv' 
VERBOSE = False

# Call the function with the relative path to delete all csv files in 1_preprocess folder.
delete_all_csv_files(r"../data/1_preprocess/")

#Get list of columns from formatted sosec code book
required_columns, questions_having_subcategories, _i, _j = get_columns_from_formatted_codebook(FILE_PATH_REFORMATED_SOSEC_CODE_BOOK,VERBOSE)

# list of column names to keep
columns_to_keep =  ['i_TIME'] + required_columns 

#Only keep the columns as per sosec dataset
output_file_path = filter_datasets_for_required_col(FILE_PATH_DATASET,FILE_PATH_REFORMATED_SOSEC_CODE_BOOK,columns_to_keep,VERBOSE)

#Check for duplicate columns in the filtered dataset
check_duplicate_cols(output_file_path)

#replace 6 and 0 in column F2A3 by empty cells
output_file_path = perform_filter_on_dataset_F2A3(output_file_path)
print("\nNo of rows and columns in original dataset after setting 6,0 to empty in F2A3")
no_of_rows = get_row_count(output_file_path)
no_of_columns = count_columns_in_csv(output_file_path)

#replace 0 in column F7mA1 (job category) by empty cells
output_file_path = perform_filter_on_dataset_F7mA1(output_file_path)
print("\nNo of rows and columns in original dataset after setting 0 to empty in F7mA1")
no_of_rows = get_row_count(output_file_path)
no_of_columns = count_columns_in_csv(output_file_path)

#Remove out of range data
output_file_path = delete_rows_for_out_of_range_data(output_file_path,FILE_PATH_REFORMATED_SOSEC_CODE_BOOK,False)
print("\nNo of data after removing out of invalid range values from dataset")
no_of_rows = get_row_count(output_file_path)
no_of_columns = count_columns_in_csv(output_file_path)


#filter rows based on F7cA1 (YOB) values, remove where out of reasonable age range.
yob_start = 1959
yob_end = 2004
output_file_path = perform_filter_on_dataset_F7cA1(output_file_path,1959,2004)
print(f"\nNo of data rows after drop rows having out of range F7cA1 (YOB:{yob_start}-{yob_end}) values")
no_of_rows = get_row_count(output_file_path)
no_of_columns = count_columns_in_csv(output_file_path)

#Delete columns having more than 70% null with exceptions of some columns
col_to_exclude =["F6a_DemPartyA2","F6a_RepPartyA2","F6b_DemPartyA2","F6b_RepPartyA2"]
output_file_path = drop_high_null_columns(output_file_path,FILE_PATH_REFORMATED_SOSEC_CODE_BOOK, col_to_exclude,0.7)
print(f"\nNo of data rows after Delete columns having more than 70% null with exceptions of columns: {col_to_exclude} ")
no_of_rows = get_row_count(output_file_path)
no_of_columns = count_columns_in_csv(output_file_path)

#Delete data which is filled in rapidly without reading by using i_time column
percentile_val = 10
output_file_path = remove_below_percentile(output_file_path,10)
print(f"\nNo of data rows and col which is filled having time of i_TIME more then {percentile_val}th percentile")
no_of_rows = get_row_count(output_file_path)
no_of_columns = count_columns_in_csv(output_file_path)

#Rename all columns in the data file to questions for creating personas.
FILE_PATH_REFORMATED_SOSEC_CODE_BOOK = r'..\data\0_Reformated_SOSEC_Code-book_US_November_Reformulated_Questions.csv' 
prepare_for_personas(output_file_path,FILE_PATH_REFORMATED_SOSEC_CODE_BOOK)


#Not applied: hot-encooding.
'''
#Encode the columns which are based on categorical values. 
#F7a:Gender
#F7bA1	Enter a 5-digit Zip number.:
#F7d	Were you born in the US?
#F7e	Was your mother born in the US?
#F7f	Was your father born in the US?
#F7g:  educational level
#F7h: employment status?
#F7i	What is your marital status?
#F7lA1	Which religious community do you belong to?
#F7mA1	To which of the following occupational groups do you belong?
#F7n	Which ethnic group do you belong to?
input_file_path = output_file_path
columns_to_encode = ['F7a', 'F7bA1','F7d','F7e','F7f','F7g', 'F7h', 'F7i', 'F7lA1', 'F7mA1','F7n']  
# Perform one-hot encoding
output_file_path = one_hot_encode_csv(input_file_path, columns_to_encode)
print("\nNo of data rows and columns after encoding and removing the encoded columns.")
no_of_rows = get_row_count(output_file_path)
no_of_columns = count_columns_in_csv(output_file_path)
'''


Deleted: ../data/1_preprocess\1_codebook_no_matching_columns_in_dataset.csv
Deleted: ../data/1_preprocess\1_df_dataset_with_codebook_columns_full_no_processing.csv
Deleted: ../data/1_preprocess\2_df_dataset_with_codebook_columns_full_F2A3.csv
Deleted: ../data/1_preprocess\3_df_dataset_with_codebook_columns_full_F7mA1.csv
Deleted: ../data/1_preprocess\4_df_dataset_with_codebook_columns_filtered_outofrange.csv
Deleted: ../data/1_preprocess\5_df_dataset_with_codebook_columns_filtered_F7cA1-yob.csv
Deleted: ../data/1_preprocess\6_df_dataset_with_codebook_columns_filtered_lessdata.csv
Deleted: ../data/1_preprocess\6_removed_columns_due_to_lessdata.csv
Deleted: ../data/1_preprocess\7_df_dataset_with_codebook_columns_filtered_itime.csv
Deleted: ../data/1_preprocess\9_processed_data_for_personas.csv




C:\Users\Jim\AppData\Local\Temp\ipykernel_35920\368653022.py:8: DtypeWarning: Columns (57,58,62,68,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_dataset)



Nice! No duplicate column titles found.

Updated dataset saved to: ../data/1_preprocess/2_df_dataset_with_codebook_columns_full_F2A3.csv

No of rows and columns in original dataset after setting 6,0 to empty in F2A3
Number of rows in the csv: 36781
Number of columns in the CSV: 96
Updated dataset saved to: ../data/1_preprocess/3_df_dataset_with_codebook_columns_full_F7mA1.csv

No of rows and columns in original dataset after setting 0 to empty in F7mA1
Number of rows in the csv: 36781
Number of columns in the CSV: 96
Cleaned data saved to '../data/1_preprocess/4_df_dataset_with_codebook_columns_filtered_outofrange.csv'.

No of data after removing out of invalid range values from dataset
Number of rows in the csv: 36198
Number of columns in the CSV: 96
Updated dataset saved to: ../data/1_preprocess/5_df_dataset_with_codebook_columns_filtered_F7cA1-yob.csv

No of data rows after drop rows having out of range F7cA1 (YOB:1959-2004) values
Number of rows in the csv: 22712
Number of columns

'\n#Encode the columns which are based on categorical values. \n#F7a:Gender\n#F7bA1\tEnter a 5-digit Zip number.:\n#F7d\tWere you born in the US?\n#F7e\tWas your mother born in the US?\n#F7f\tWas your father born in the US?\n#F7g:  educational level\n#F7h: employment status?\n#F7i\tWhat is your marital status?\n#F7lA1\tWhich religious community do you belong to?\n#F7mA1\tTo which of the following occupational groups do you belong?\n#F7n\tWhich ethnic group do you belong to?\ninput_file_path = output_file_path\ncolumns_to_encode = [\'F7a\', \'F7bA1\',\'F7d\',\'F7e\',\'F7f\',\'F7g\', \'F7h\', \'F7i\', \'F7lA1\', \'F7mA1\',\'F7n\']  \n# Perform one-hot encoding\noutput_file_path = one_hot_encode_csv(input_file_path, columns_to_encode)\nprint("\nNo of data rows and columns after encoding and removing the encoded columns.")\nno_of_rows = get_row_count(output_file_path)\nno_of_columns = count_columns_in_csv(output_file_path)\n'